# Lab 1:  Data Loading, GPU Dataframe Creation, and Data Manipulation
Thanks to Anaconda for some material

In this lab you will learn how to 
- Load data into a GPU Data Frame (GDF)
- Manitulate data in a GDF and performance basic ETL functions


As you progress in this lab look for instances of ***TASK:***, this will be where you will be asked to take an action to complete this lab.

***TASK:*** Execute the cell below to auto-time execution of every cell

In [3]:
# Add autotime of each block
!pip install ipython-autotime
%load_ext autotime

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


<br>
## Loading Data
The file we are going to load is netflow1.csv

### Traditional interface through Pandas

In [4]:
import pandas as pd

time: 199 ms


In [6]:
# let's define the data - column snames and data types
cols = [
     "strdate",
    "srcip",
    "dstip",
    "srcport",
    "dstport",
    "srcbytes",
    "dstbytes"   
]


dtypes = {    
    "strdate"  : str,
    "srcip"    : str,
    "dstip"    : str,
    "srcport"  : int,
    "dstport"  : int,
    "srcbytes" : int,
    "dstbytes" : int

}

time: 8.48 ms


In [7]:
file_1 = '../data/netflow1.csv'
file_1

'../data/netflow1.csv'

time: 12.5 ms


In [16]:
net1_df = pd.read_csv(file_1,  names=cols, dtype=dtypes, skiprows=1)

time: 9.64 s


In [17]:
net1_df.dtypes

strdate     object
srcip       object
dstip       object
srcport      int64
dstport      int64
srcbytes     int64
dstbytes     int64
dtype: object

time: 6.69 ms


<br>
## Creating a GPU Dataframe


In [18]:
import pygdf

time: 1.17 ms


In [19]:
gdf = pygdf.DataFrame.from_pandas(net1_df)

time: 792 ms


In [20]:
gdf.dtypes

strdate     object
srcip       object
dstip       object
srcport      int64
dstport      int64
srcbytes     int64
dstbytes     int64
dtype: object

time: 5.46 ms


<br>
## Column Functionals and Transformations
One of the basic GDF operations is column transform. We can perform built-in arithmetic operations on each column, such as type casting:

### Accessing a Column

In [22]:
gdf['srcport'].head()

      
0  137
1  137
2 5040
3 5067
4 8749

time: 8.95 ms


<br>
***Tasks*** Find the largest amount of data sent to a distination

<details><summary>Answer (one possible)</summary>
<p>
gdf['dstbytes'].max()
</p>
</details>